# Models development (definitive version)

In this notebook we make models for clusters with unimodal fire season using the delta time series.

We consider linear models, random forests y k-nearest neighbours. For each cluster we develop 6 models using each of these techniques: one for each lag between 0 and 3, another one using lags 1, 2 and 3 and the final one using the four available lags.

We use the functions defined in the file "scripts/modelling_functions_new.R".

In [2]:
setwd("..")

In [3]:
source("scripts/modelling_functions_new.R")

Warning message:
"package 'tidyr' was built under R version 3.6.3"Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: sp
Warning message:
"package 'sp' was built under R version 3.6.3"Loading required package: magrittr
Warning message:
"package 'magrittr' was built under R version 3.6.3"
Attaching package: 'magrittr'

The following object is masked from 'package:tidyr':

    extract

Warning message:
"package 'caret' was built under R version 3.6.3"Loading required package: lattice
Warning message:
"package 'lattice' was built under R version 3.6.3"Warning message:
"package 'randomForest' was built under R version 3.6.3"randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: 'randomForest'

The following object is m

In [4]:
# Data required
library('ggplot2')
load('data/fireSeason_def_2.Rdata', verbose = T)
load("data/Fire/ba_mon_time_series_masked.Rdata", verbose = T)
load("data/Fire/ba_mon_clim_masked_df.Rdata", verbose = T)
load('data/Correlation/corrDfDeltaAnnualClusV2.Rdata', verbose = T)
load('data/Correlation/corrDfDeltaAnnualClusLag1V2.Rdata', verbose = T)
load('data/Correlation/corrDfDeltaAnnualClusLag2V2.Rdata', verbose = T)
load('data/Correlation/corrDfDeltaAnnualClusLag3V2.Rdata', verbose = T)

Loading objects:
  fireSeason_def_2
Loading objects:
  masked_coords
  dates
  masked_ba_series
Loading objects:
  df_masked
  masked_coords
Loading objects:
  corr.df.delta
Loading objects:
  corr.df.delta.1
Loading objects:
  corr.df.delta.2
Loading objects:
  corr.df.delta.3


## Climate indexes data loading

In [5]:
nina34 = read.table("data/CPC/nina34.anom.data.txt", skip = 3, nrows = 72)
colnames(nina34) = c("Year", month.abb)
nina34[,-1] = (nina34[,-1] - mean(as.matrix(nina34[32:61,-1]))) / sd(as.matrix(nina34[32:61,-1]))

In [6]:
nao = read.table("data/CPC/nao_index.tim.txt", skip = 8, header = T)
nao <- nao %>% spread(MONTH,INDEX)
colnames(nao) = c("Year", month.abb)

In [7]:
poleur = read.table("data/CPC/poleur_index.tim", skip = 8, header = T)
poleur <- poleur %>% spread(MONTH,INDEX)
colnames(poleur) = c("Year", month.abb)

In [8]:
ea = read.table("data/CPC/ea_index.tim.txt", skip = 8, header = T)
ea <- ea %>% spread(MONTH,INDEX)
colnames(ea) = c("Year", month.abb)

In [9]:
eawr = read.table("data/CPC/eawr_index.tim.txt", skip = 8, header = T)
eawr <- eawr %>% spread(MONTH,INDEX)
colnames(eawr) = c("Year", month.abb)

In [10]:
pna = read.table("data/CPC/pna_index.tim.txt", skip = 8, header = T)
pna <- pna %>% spread(MONTH,INDEX)
colnames(pna) = c("Year", month.abb)

In [11]:
scand = read.table("data/CPC/scand_index.tim.txt", skip = 8, header = T)
scand <- scand %>% spread(MONTH,INDEX)
colnames(scand) = c("Year", month.abb)

In [12]:
wp = read.table("data/CPC/wp_index.tim.txt", skip = 8, header = T)
wp <- wp %>% spread(MONTH,INDEX)
colnames(wp) = c("Year", month.abb)

# Models with lag 0

In [13]:
lm.list <- lm.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', useDeltas = T)
lm.list$results

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,lm.acc.t2,lm.acc.t3
2,1,1,3,86.12356,3.39932664,0.6091194,0.024138453,0.5285243,0.4444444,0.5000000,0.3333333,0.5000000
3,1,2,2,73.19566,-2.35847383,0.5546609,0.002693328,0.6632875,0.5000000,0.5000000,0.3333333,0.6666667
4,1,3,1,89.27483,0.40332848,0.3739380,0.072363808,0.4334172,0.5000000,0.5000000,0.3333333,0.6666667
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,2,81.10800,3.00950653,0.4748432,0.014466709,0.5654238,0.5000000,0.6666667,0.3333333,0.5000000
7,1,6,1,92.90298,1.68746230,0.2224139,0.184705134,0.3274423,0.5000000,0.5000000,0.5000000,0.5000000
8,2,1,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
10,2,3,2,73.25361,3.77553676,0.6090327,0.002455416,0.6678372,0.6111111,0.6666667,0.6666667,0.5000000
11,2,4,0,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [14]:
rf.list <- rf.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', useDeltas = T)
rf.list$results

,biome,cluster,rf.mtry,rf.Ntree,rf.maxnodes,rf.RMSE,rf.bias,rf.RVar,rf.cor.pvalue,rf.cor,rf.acc,rf.acc.t1,rf.acc.t2,rf.acc.t3
2,1,1,1,10,5,100.72048,-9.38461083,0.06469831,0.979774007,0.006239799,0.5263158,0.5000000,0.5000000,0.5714286
3,1,2,8,25,9,101.06332,-0.51254816,0.26216460,0.461521437,0.179748451,0.6315789,0.5000000,0.8333333,0.5714286
4,1,3,2,175,9,106.29238,0.58052764,0.08808394,0.471135505,-0.175974175,0.4210526,0.3333333,0.6666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,2,75,5,103.80098,0.02244286,0.07846591,0.668623277,-0.105058489,0.3157895,0.1666667,0.5000000,0.2857143
7,1,6,7,75,5,120.96137,-2.62541771,0.17828486,0.064230260,-0.432741688,0.2105263,0.3333333,0.1666667,0.1428571
8,2,1,8,125,7,98.14574,7.62037444,0.28730119,0.306848156,0.255155295,0.4444444,0.5000000,0.3333333,0.5000000
9,2,2,7,175,5,117.80297,-7.33488342,0.12064048,0.031586063,-0.481511378,0.2500000,0.1428571,0.5000000,0.1428571
10,2,3,8,10,11,98.18923,-5.88486672,0.25981591,0.320030964,0.241109410,0.5789474,0.5000000,0.6666667,0.5714286
11,2,4,2,25,7,91.10674,3.24659613,0.12926893,0.137577123,0.353540873,0.5789474,0.5000000,0.5000000,0.7142857


In [15]:
knn.list <- knn.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', useDeltas = T)
knn.list$results

,biome,cluster,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,2,137.14728,-14.781768774,0.36015516,0.028612410,-0.50173786,0.26315789,0.1666667,0.3333333,0.2857143
3,1,2,3,95.09528,2.833708954,0.24112970,0.224008423,0.29266542,0.31578947,0.3333333,0.1666667,0.4285714
4,1,3,4,100.61996,-2.395520903,0.22788217,0.493397350,0.16737940,0.26315789,0.3333333,0.1666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,5,104.13405,1.210230525,0.15670738,0.949961205,0.01544487,0.31578947,0.1666667,0.3333333,0.4285714
7,1,6,3,113.64576,-8.031465158,0.33622634,0.943429585,-0.01746381,0.26315789,0.3333333,0.1666667,0.2857143
8,2,1,2,114.96070,30.201924211,0.44823218,0.667831984,0.10864527,0.33333333,0.3333333,0.3333333,0.3333333
9,2,2,5,108.49027,-2.625828192,0.04859062,0.058336892,-0.43016616,0.25000000,0.2857143,0.3333333,0.1428571
10,2,3,2,107.82115,3.649410612,0.63421665,0.289190824,0.25647202,0.26315789,0.3333333,0.1666667,0.2857143
11,2,4,3,97.48834,-3.700316635,0.26393444,0.291736624,0.25516735,0.63157895,0.3333333,0.8333333,0.7142857


In [16]:
table = cbind(lm.list$results, rf.list$results[,-c(1,2)], knn.list$results[,-c(1,2)])
table

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,...,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,3,86.12356,3.39932664,0.6091194,0.024138453,0.5285243,0.4444444,0.5000000,...,2,137.14728,-14.781768774,0.36015516,0.028612410,-0.50173786,0.26315789,0.1666667,0.3333333,0.2857143
3,1,2,2,73.19566,-2.35847383,0.5546609,0.002693328,0.6632875,0.5000000,0.5000000,...,3,95.09528,2.833708954,0.24112970,0.224008423,0.29266542,0.31578947,0.3333333,0.1666667,0.4285714
4,1,3,1,89.27483,0.40332848,0.3739380,0.072363808,0.4334172,0.5000000,0.5000000,...,4,100.61996,-2.395520903,0.22788217,0.493397350,0.16737940,0.26315789,0.3333333,0.1666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,2,81.10800,3.00950653,0.4748432,0.014466709,0.5654238,0.5000000,0.6666667,...,5,104.13405,1.210230525,0.15670738,0.949961205,0.01544487,0.31578947,0.1666667,0.3333333,0.4285714
7,1,6,1,92.90298,1.68746230,0.2224139,0.184705134,0.3274423,0.5000000,0.5000000,...,3,113.64576,-8.031465158,0.33622634,0.943429585,-0.01746381,0.26315789,0.3333333,0.1666667,0.2857143
8,2,1,0,NA,NA,NA,NA,NA,NA,NA,...,2,114.96070,30.201924211,0.44823218,0.667831984,0.10864527,0.33333333,0.3333333,0.3333333,0.3333333
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,...,5,108.49027,-2.625828192,0.04859062,0.058336892,-0.43016616,0.25000000,0.2857143,0.3333333,0.1428571
10,2,3,2,73.25361,3.77553676,0.6090327,0.002455416,0.6678372,0.6111111,0.6666667,...,2,107.82115,3.649410612,0.63421665,0.289190824,0.25647202,0.26315789,0.3333333,0.1666667,0.2857143
11,2,4,0,NA,NA,NA,NA,NA,NA,NA,...,3,97.48834,-3.700316635,0.26393444,0.291736624,0.25516735,0.63157895,0.3333333,0.8333333,0.7142857


In [17]:
save(table, file = 'data/ModelData/models_results_per_cluster_with_delta_no_persistence_v2.Rdata')

In [19]:
# Clusters that have a linear, a random forest and a knn models
table1 <- table[-which(is.na(table$lm.RMSE)),]
table1

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,...,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,3,86.12356,3.39932664,0.6091194,0.024138453,0.5285243,0.4444444,0.5000000,...,2,137.14728,-14.7817688,0.3601552,0.02861241,-0.50173786,0.2631579,0.1666667,0.3333333,0.2857143
3,1,2,2,73.19566,-2.35847383,0.5546609,0.002693328,0.6632875,0.5000000,0.5000000,...,3,95.09528,2.8337090,0.2411297,0.22400842,0.29266542,0.3157895,0.3333333,0.1666667,0.4285714
4,1,3,1,89.27483,0.40332848,0.3739380,0.072363808,0.4334172,0.5000000,0.5000000,...,4,100.61996,-2.3955209,0.2278822,0.49339735,0.16737940,0.2631579,0.3333333,0.1666667,0.2857143
6,1,5,2,81.10800,3.00950653,0.4748432,0.014466709,0.5654238,0.5000000,0.6666667,...,5,104.13405,1.2102305,0.1567074,0.94996120,0.01544487,0.3157895,0.1666667,0.3333333,0.4285714
7,1,6,1,92.90298,1.68746230,0.2224139,0.184705134,0.3274423,0.5000000,0.5000000,...,3,113.64576,-8.0314652,0.3362263,0.94342958,-0.01746381,0.2631579,0.3333333,0.1666667,0.2857143
10,2,3,2,73.25361,3.77553676,0.6090327,0.002455416,0.6678372,0.6111111,0.6666667,...,2,107.82115,3.6494106,0.6342167,0.28919082,0.25647202,0.2631579,0.3333333,0.1666667,0.2857143
12,2,5,1,88.74507,0.18008557,0.4205557,0.059473173,0.4523211,0.3888889,0.5000000,...,2,88.99993,-13.9543086,0.2439070,0.06359433,0.43367415,0.4210526,0.6666667,0.1666667,0.4285714
13,2,6,1,81.19586,-2.08804540,0.4815469,0.014586932,0.5648583,0.6111111,0.5000000,...,1,123.13124,-27.7557720,1.1070227,0.24664763,0.27941857,0.3157895,0.1666667,0.1666667,0.5714286
16,4,1,1,92.12212,2.40506686,0.3049364,0.132396980,0.3685008,0.5000000,0.6666667,...,3,122.87256,5.0090245,0.3340978,0.36050761,-0.22222392,0.4736842,0.3333333,0.6666667,0.4285714
21,5,1,1,85.41103,0.85776247,0.3927945,0.036749643,0.4949921,0.6111111,0.6666667,...,1,138.74172,13.5601322,0.8039915,0.63555778,-0.11624642,0.2631579,0.3333333,0.1666667,0.2857143


# With 1 month persistence

In [20]:
lm.list <- lm.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta.1, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 1, useDeltas = T)
lm.list$results

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,lm.acc.t2,lm.acc.t3
2,1,1,1,85.74284,2.5111926,0.3440181,0.0424534937,0.4827168,0.3888889,0.6666667,0.3333333,0.1666667
3,1,2,2,67.61227,-2.9702442,0.6189304,0.0007185666,0.7218769,0.6111111,0.6666667,0.5000000,0.6666667
4,1,3,1,84.90482,0.4044526,0.4132092,0.0323345778,0.5055423,0.6111111,0.6666667,0.5000000,0.6666667
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,1,64.85576,0.6770239,0.6374318,0.0003722483,0.7465685,0.5555556,0.5000000,0.5000000,0.6666667
7,1,6,1,90.36465,2.7408328,0.2789868,0.1066567157,0.3930117,0.5555556,0.5000000,0.5000000,0.6666667
8,2,1,2,83.08598,0.2773470,0.5040408,0.0243908728,0.5426907,0.5294118,0.5000000,0.4000000,0.6666667
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
10,2,3,2,80.37352,3.4068340,0.5195197,0.0115488459,0.5804560,0.4444444,0.5000000,0.3333333,0.5000000
11,2,4,1,91.87663,0.1093478,0.2328123,0.1528430649,0.3513191,0.4444444,0.6666667,0.3333333,0.3333333


In [21]:
rf.list <- rf.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta.1, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 1, useDeltas = T)
rf.list$results

,biome,cluster,rf.mtry,rf.Ntree,rf.maxnodes,rf.RMSE,rf.bias,rf.RVar,rf.cor.pvalue,rf.cor,rf.acc,rf.acc.t1,rf.acc.t2,rf.acc.t3
2,1,1,1,10,5,100.72048,-9.38461083,0.06469831,0.979774007,0.006239799,0.5263158,0.5000000,0.5000000,0.5714286
3,1,2,8,25,9,101.06332,-0.51254816,0.26216460,0.461521437,0.179748451,0.6315789,0.5000000,0.8333333,0.5714286
4,1,3,2,175,9,106.29238,0.58052764,0.08808394,0.471135505,-0.175974175,0.4210526,0.3333333,0.6666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,2,75,5,103.80098,0.02244286,0.07846591,0.668623277,-0.105058489,0.3157895,0.1666667,0.5000000,0.2857143
7,1,6,7,75,5,120.96137,-2.62541771,0.17828486,0.064230260,-0.432741688,0.2105263,0.3333333,0.1666667,0.1428571
8,2,1,8,125,7,98.14574,7.62037444,0.28730119,0.306848156,0.255155295,0.4444444,0.5000000,0.3333333,0.5000000
9,2,2,7,175,5,117.80297,-7.33488342,0.12064048,0.031586063,-0.481511378,0.2500000,0.1428571,0.5000000,0.1428571
10,2,3,8,10,11,98.18923,-5.88486672,0.25981591,0.320030964,0.241109410,0.5789474,0.5000000,0.6666667,0.5714286
11,2,4,2,25,7,91.10674,3.24659613,0.12926893,0.137577123,0.353540873,0.5789474,0.5000000,0.5000000,0.7142857


In [22]:
knn.list <- knn.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta.1, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 1, useDeltas = T)
knn.list$results

,biome,cluster,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,2,137.14728,-14.781768774,0.36015516,0.028612410,-0.50173786,0.26315789,0.1666667,0.3333333,0.2857143
3,1,2,3,95.09528,2.833708954,0.24112970,0.224008423,0.29266542,0.31578947,0.3333333,0.1666667,0.4285714
4,1,3,4,100.61996,-2.395520903,0.22788217,0.493397350,0.16737940,0.26315789,0.3333333,0.1666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,5,104.13405,1.210230525,0.15670738,0.949961205,0.01544487,0.31578947,0.1666667,0.3333333,0.4285714
7,1,6,3,113.64576,-8.031465158,0.33622634,0.943429585,-0.01746381,0.26315789,0.3333333,0.1666667,0.2857143
8,2,1,2,114.96070,30.201924211,0.44823218,0.667831984,0.10864527,0.33333333,0.3333333,0.3333333,0.3333333
9,2,2,5,108.49027,-2.625828192,0.04859062,0.058336892,-0.43016616,0.25000000,0.2857143,0.3333333,0.1428571
10,2,3,2,107.82115,3.649410612,0.63421665,0.289190824,0.25647202,0.26315789,0.3333333,0.1666667,0.2857143
11,2,4,3,97.48834,-3.700316635,0.26393444,0.291736624,0.25516735,0.63157895,0.3333333,0.8333333,0.7142857


In [23]:
table = cbind(lm.list$results, rf.list$results[,-c(1,2)], knn.list$results[,-c(1,2)])
table

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,...,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,1,85.74284,2.5111926,0.3440181,0.0424534937,0.4827168,0.3888889,0.6666667,...,2,137.14728,-14.781768774,0.36015516,0.028612410,-0.50173786,0.26315789,0.1666667,0.3333333,0.2857143
3,1,2,2,67.61227,-2.9702442,0.6189304,0.0007185666,0.7218769,0.6111111,0.6666667,...,3,95.09528,2.833708954,0.24112970,0.224008423,0.29266542,0.31578947,0.3333333,0.1666667,0.4285714
4,1,3,1,84.90482,0.4044526,0.4132092,0.0323345778,0.5055423,0.6111111,0.6666667,...,4,100.61996,-2.395520903,0.22788217,0.493397350,0.16737940,0.26315789,0.3333333,0.1666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,1,64.85576,0.6770239,0.6374318,0.0003722483,0.7465685,0.5555556,0.5000000,...,5,104.13405,1.210230525,0.15670738,0.949961205,0.01544487,0.31578947,0.1666667,0.3333333,0.4285714
7,1,6,1,90.36465,2.7408328,0.2789868,0.1066567157,0.3930117,0.5555556,0.5000000,...,3,113.64576,-8.031465158,0.33622634,0.943429585,-0.01746381,0.26315789,0.3333333,0.1666667,0.2857143
8,2,1,2,83.08598,0.2773470,0.5040408,0.0243908728,0.5426907,0.5294118,0.5000000,...,2,114.96070,30.201924211,0.44823218,0.667831984,0.10864527,0.33333333,0.3333333,0.3333333,0.3333333
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,...,5,108.49027,-2.625828192,0.04859062,0.058336892,-0.43016616,0.25000000,0.2857143,0.3333333,0.1428571
10,2,3,2,80.37352,3.4068340,0.5195197,0.0115488459,0.5804560,0.4444444,0.5000000,...,2,107.82115,3.649410612,0.63421665,0.289190824,0.25647202,0.26315789,0.3333333,0.1666667,0.2857143
11,2,4,1,91.87663,0.1093478,0.2328123,0.1528430649,0.3513191,0.4444444,0.6666667,...,3,97.48834,-3.700316635,0.26393444,0.291736624,0.25516735,0.63157895,0.3333333,0.8333333,0.7142857


In [24]:
table.deltas.Lag.1 = table
save(table.deltas.Lag.1, file = 'data/ModelData/models_results_per_cluster_with_delta_lag1_v2.Rdata')

In [25]:
# Clusters that have a linear, a random forest and a knn models
table1 <- table[-which(is.na(table$lm.RMSE)),]
table1

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,...,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,1,85.74284,2.5111926,0.3440181,0.0424534937,0.4827168,0.3888889,0.6666667,...,2,137.14728,-14.7817688,0.36015516,0.02861241,-0.50173786,0.26315789,0.1666667,0.3333333,0.2857143
3,1,2,2,67.61227,-2.9702442,0.6189304,0.0007185666,0.7218769,0.6111111,0.6666667,...,3,95.09528,2.8337090,0.24112970,0.22400842,0.29266542,0.31578947,0.3333333,0.1666667,0.4285714
4,1,3,1,84.90482,0.4044526,0.4132092,0.0323345778,0.5055423,0.6111111,0.6666667,...,4,100.61996,-2.3955209,0.22788217,0.49339735,0.16737940,0.26315789,0.3333333,0.1666667,0.2857143
6,1,5,1,64.85576,0.6770239,0.6374318,0.0003722483,0.7465685,0.5555556,0.5000000,...,5,104.13405,1.2102305,0.15670738,0.94996120,0.01544487,0.31578947,0.1666667,0.3333333,0.4285714
7,1,6,1,90.36465,2.7408328,0.2789868,0.1066567157,0.3930117,0.5555556,0.5000000,...,3,113.64576,-8.0314652,0.33622634,0.94342958,-0.01746381,0.26315789,0.3333333,0.1666667,0.2857143
8,2,1,2,83.08598,0.2773470,0.5040408,0.0243908728,0.5426907,0.5294118,0.5000000,...,2,114.96070,30.2019242,0.44823218,0.66783198,0.10864527,0.33333333,0.3333333,0.3333333,0.3333333
10,2,3,2,80.37352,3.4068340,0.5195197,0.0115488459,0.5804560,0.4444444,0.5000000,...,2,107.82115,3.6494106,0.63421665,0.28919082,0.25647202,0.26315789,0.3333333,0.1666667,0.2857143
11,2,4,1,91.87663,0.1093478,0.2328123,0.1528430649,0.3513191,0.4444444,0.6666667,...,3,97.48834,-3.7003166,0.26393444,0.29173662,0.25516735,0.63157895,0.3333333,0.8333333,0.7142857
12,2,5,1,88.23122,0.1130856,0.3492651,0.0648003801,0.4441717,0.2777778,0.3333333,...,2,88.99993,-13.9543086,0.24390704,0.06359433,0.43367415,0.42105263,0.6666667,0.1666667,0.4285714
13,2,6,2,93.43931,-1.8791368,0.3675765,0.1355274894,0.3657559,0.5000000,0.3333333,...,1,123.13124,-27.7557720,1.10702267,0.24664763,0.27941857,0.31578947,0.1666667,0.1666667,0.5714286


# With 2-months persistence

In [26]:
lm.list <- lm.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta.2, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 2, useDeltas = T)
lm.list$results

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,lm.acc.t2,lm.acc.t3
2,1,1,1,91.73388,-1.5466423,0.2605746,0.1396178017,0.3622352,0.4444444,0.3333333,0.3333333,0.6666667
3,1,2,2,79.76404,-2.9267446,0.4704567,0.0113641158,0.5815061,0.5000000,0.6666667,0.5000000,0.3333333
4,1,3,2,85.21115,-1.3124034,0.5453157,0.0249853248,0.5258904,0.5555556,0.5000000,0.5000000,0.6666667
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,1,64.98062,0.5348240,0.6197156,0.0003910237,0.7448105,0.6111111,0.6666667,0.5000000,0.6666667
7,1,6,2,88.64282,1.8505484,0.3602208,0.0674025860,0.4403713,0.5000000,0.5000000,0.3333333,0.6666667
8,2,1,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
10,2,3,2,72.61800,4.2441275,0.6315470,0.0020537681,0.6764175,0.5555556,0.5000000,0.5000000,0.6666667
11,2,4,0,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [27]:
rf.list <- rf.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta.2, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 2, useDeltas = T)
rf.list$results

,biome,cluster,rf.mtry,rf.Ntree,rf.maxnodes,rf.RMSE,rf.bias,rf.RVar,rf.cor.pvalue,rf.cor,rf.acc,rf.acc.t1,rf.acc.t2,rf.acc.t3
2,1,1,1,10,5,100.72048,-9.38461083,0.06469831,0.979774007,0.006239799,0.5263158,0.5000000,0.5000000,0.5714286
3,1,2,8,25,9,101.06332,-0.51254816,0.26216460,0.461521437,0.179748451,0.6315789,0.5000000,0.8333333,0.5714286
4,1,3,2,175,9,106.29238,0.58052764,0.08808394,0.471135505,-0.175974175,0.4210526,0.3333333,0.6666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,2,75,5,103.80098,0.02244286,0.07846591,0.668623277,-0.105058489,0.3157895,0.1666667,0.5000000,0.2857143
7,1,6,7,75,5,120.96137,-2.62541771,0.17828486,0.064230260,-0.432741688,0.2105263,0.3333333,0.1666667,0.1428571
8,2,1,8,125,7,98.14574,7.62037444,0.28730119,0.306848156,0.255155295,0.4444444,0.5000000,0.3333333,0.5000000
9,2,2,7,175,5,117.80297,-7.33488342,0.12064048,0.031586063,-0.481511378,0.2500000,0.1428571,0.5000000,0.1428571
10,2,3,8,10,11,98.18923,-5.88486672,0.25981591,0.320030964,0.241109410,0.5789474,0.5000000,0.6666667,0.5714286
11,2,4,2,25,7,91.10674,3.24659613,0.12926893,0.137577123,0.353540873,0.5789474,0.5000000,0.5000000,0.7142857


In [28]:
knn.list <- knn.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta.2, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 2, useDeltas = T)
knn.list$results

,biome,cluster,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,2,137.14728,-14.781768774,0.36015516,0.028612410,-0.50173786,0.26315789,0.1666667,0.3333333,0.2857143
3,1,2,3,95.09528,2.833708954,0.24112970,0.224008423,0.29266542,0.31578947,0.3333333,0.1666667,0.4285714
4,1,3,4,100.61996,-2.395520903,0.22788217,0.493397350,0.16737940,0.26315789,0.3333333,0.1666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,5,104.13405,1.210230525,0.15670738,0.949961205,0.01544487,0.31578947,0.1666667,0.3333333,0.4285714
7,1,6,3,113.64576,-8.031465158,0.33622634,0.943429585,-0.01746381,0.26315789,0.3333333,0.1666667,0.2857143
8,2,1,2,114.96070,30.201924211,0.44823218,0.667831984,0.10864527,0.33333333,0.3333333,0.3333333,0.3333333
9,2,2,5,108.49027,-2.625828192,0.04859062,0.058336892,-0.43016616,0.25000000,0.2857143,0.3333333,0.1428571
10,2,3,2,107.82115,3.649410612,0.63421665,0.289190824,0.25647202,0.26315789,0.3333333,0.1666667,0.2857143
11,2,4,3,97.48834,-3.700316635,0.26393444,0.291736624,0.25516735,0.63157895,0.3333333,0.8333333,0.7142857


In [29]:
table = cbind(lm.list$results, rf.list$results[,-c(1,2)], knn.list$results[,-c(1,2)])
table

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,...,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,1,91.73388,-1.5466423,0.2605746,0.1396178017,0.3622352,0.4444444,0.3333333,...,2,137.14728,-14.781768774,0.36015516,0.028612410,-0.50173786,0.26315789,0.1666667,0.3333333,0.2857143
3,1,2,2,79.76404,-2.9267446,0.4704567,0.0113641158,0.5815061,0.5000000,0.6666667,...,3,95.09528,2.833708954,0.24112970,0.224008423,0.29266542,0.31578947,0.3333333,0.1666667,0.4285714
4,1,3,2,85.21115,-1.3124034,0.5453157,0.0249853248,0.5258904,0.5555556,0.5000000,...,4,100.61996,-2.395520903,0.22788217,0.493397350,0.16737940,0.26315789,0.3333333,0.1666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,1,64.98062,0.5348240,0.6197156,0.0003910237,0.7448105,0.6111111,0.6666667,...,5,104.13405,1.210230525,0.15670738,0.949961205,0.01544487,0.31578947,0.1666667,0.3333333,0.4285714
7,1,6,2,88.64282,1.8505484,0.3602208,0.0674025860,0.4403713,0.5000000,0.5000000,...,3,113.64576,-8.031465158,0.33622634,0.943429585,-0.01746381,0.26315789,0.3333333,0.1666667,0.2857143
8,2,1,0,NA,NA,NA,NA,NA,NA,NA,...,2,114.96070,30.201924211,0.44823218,0.667831984,0.10864527,0.33333333,0.3333333,0.3333333,0.3333333
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,...,5,108.49027,-2.625828192,0.04859062,0.058336892,-0.43016616,0.25000000,0.2857143,0.3333333,0.1428571
10,2,3,2,72.61800,4.2441275,0.6315470,0.0020537681,0.6764175,0.5555556,0.5000000,...,2,107.82115,3.649410612,0.63421665,0.289190824,0.25647202,0.26315789,0.3333333,0.1666667,0.2857143
11,2,4,0,NA,NA,NA,NA,NA,NA,NA,...,3,97.48834,-3.700316635,0.26393444,0.291736624,0.25516735,0.63157895,0.3333333,0.8333333,0.7142857


In [30]:
table.deltas.Lag.2 = table
save(table.deltas.Lag.2, file = 'data/ModelData/models_results_per_cluster_with_delta_lag2_v2.Rdata')

In [31]:
# Clusters that have a linear, a random forest and a knn models
table1 <- table[-which(is.na(table$lm.RMSE)),]
table1

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,...,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,1,91.73388,-1.5466423,0.2605746,0.1396178017,0.3622352,0.4444444,0.3333333,...,2,137.14728,-14.781768774,0.36015516,0.02861241,-0.50173786,0.26315789,0.1666667,0.3333333,0.2857143
3,1,2,2,79.76404,-2.9267446,0.4704567,0.0113641158,0.5815061,0.5000000,0.6666667,...,3,95.09528,2.833708954,0.24112970,0.22400842,0.29266542,0.31578947,0.3333333,0.1666667,0.4285714
4,1,3,2,85.21115,-1.3124034,0.5453157,0.0249853248,0.5258904,0.5555556,0.5000000,...,4,100.61996,-2.395520903,0.22788217,0.49339735,0.16737940,0.26315789,0.3333333,0.1666667,0.2857143
6,1,5,1,64.98062,0.5348240,0.6197156,0.0003910237,0.7448105,0.6111111,0.6666667,...,5,104.13405,1.210230525,0.15670738,0.94996120,0.01544487,0.31578947,0.1666667,0.3333333,0.4285714
7,1,6,2,88.64282,1.8505484,0.3602208,0.0674025860,0.4403713,0.5000000,0.5000000,...,3,113.64576,-8.031465158,0.33622634,0.94342958,-0.01746381,0.26315789,0.3333333,0.1666667,0.2857143
10,2,3,2,72.61800,4.2441275,0.6315470,0.0020537681,0.6764175,0.5555556,0.5000000,...,2,107.82115,3.649410612,0.63421665,0.28919082,0.25647202,0.26315789,0.3333333,0.1666667,0.2857143
12,2,5,1,101.07726,-3.2829654,0.3092348,0.4131902715,0.2055579,0.5000000,0.3333333,...,2,88.99993,-13.954308620,0.24390704,0.06359433,0.43367415,0.42105263,0.6666667,0.1666667,0.4285714
13,2,6,2,93.06519,-1.0874957,0.4107957,0.1143675501,0.3852662,0.5000000,0.5000000,...,1,123.13124,-27.755772001,1.10702267,0.24664763,0.27941857,0.31578947,0.1666667,0.1666667,0.5714286
15,3,2,1,96.16524,-2.4511758,0.2687768,0.2605866873,0.2799174,0.3333333,0.5000000,...,5,107.69407,8.665082383,0.05752403,0.16625812,-0.33102076,0.47368421,0.3333333,0.6666667,0.4285714
16,4,1,1,82.91826,-0.9535445,0.4460557,0.0213766430,0.5376496,0.5000000,0.5000000,...,3,122.87256,5.009024500,0.33409785,0.36050761,-0.22222392,0.47368421,0.3333333,0.6666667,0.4285714


# With 3-months persistence

In [32]:
lm.list <- lm.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta.3, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 3, useDeltas = T)
lm.list$results

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,lm.acc.t2,lm.acc.t3
2,1,1,2,78.07489,1.4228224,0.6027327,0.0064142416,0.6166746,0.5000000,0.6666667,0.3333333,0.5000000
3,1,2,3,80.79492,4.7475213,0.5928941,0.0104225401,0.5870807,0.5000000,0.6666667,0.3333333,0.5000000
4,1,3,1,70.70616,-0.1757042,0.5703419,0.0015566525,0.6892131,0.7222222,0.6666667,0.6666667,0.8333333
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,3,93.44264,0.7528499,0.5339227,0.0851030136,0.4170440,0.5000000,0.6666667,0.3333333,0.5000000
7,1,6,3,71.14740,-2.5591976,0.6870041,0.0013743676,0.6947676,0.5000000,0.6666667,0.3333333,0.5000000
8,2,1,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
10,2,3,2,66.70432,-0.4334236,0.6691557,0.0005491549,0.7322905,0.6666667,0.8333333,0.5000000,0.6666667
11,2,4,0,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [33]:
rf.list <- rf.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta.3, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 3, useDeltas = T)
rf.list$results

,biome,cluster,rf.mtry,rf.Ntree,rf.maxnodes,rf.RMSE,rf.bias,rf.RVar,rf.cor.pvalue,rf.cor,rf.acc,rf.acc.t1,rf.acc.t2,rf.acc.t3
2,1,1,1,10,5,100.72048,-9.38461083,0.06469831,0.979774007,0.006239799,0.5263158,0.5000000,0.5000000,0.5714286
3,1,2,8,25,9,101.06332,-0.51254816,0.26216460,0.461521437,0.179748451,0.6315789,0.5000000,0.8333333,0.5714286
4,1,3,2,175,9,106.29238,0.58052764,0.08808394,0.471135505,-0.175974175,0.4210526,0.3333333,0.6666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,2,75,5,103.80098,0.02244286,0.07846591,0.668623277,-0.105058489,0.3157895,0.1666667,0.5000000,0.2857143
7,1,6,7,75,5,120.96137,-2.62541771,0.17828486,0.064230260,-0.432741688,0.2105263,0.3333333,0.1666667,0.1428571
8,2,1,8,125,7,98.14574,7.62037444,0.28730119,0.306848156,0.255155295,0.4444444,0.5000000,0.3333333,0.5000000
9,2,2,7,175,5,117.80297,-7.33488342,0.12064048,0.031586063,-0.481511378,0.2500000,0.1428571,0.5000000,0.1428571
10,2,3,8,10,11,98.18923,-5.88486672,0.25981591,0.320030964,0.241109410,0.5789474,0.5000000,0.6666667,0.5714286
11,2,4,2,25,7,91.10674,3.24659613,0.12926893,0.137577123,0.353540873,0.5789474,0.5000000,0.5000000,0.7142857


In [34]:
knn.list <- knn.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta.3,
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 3, useDeltas = T)
knn.list$results

,biome,cluster,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,2,137.14728,-14.781768774,0.36015516,0.028612410,-0.50173786,0.26315789,0.1666667,0.3333333,0.2857143
3,1,2,3,95.09528,2.833708954,0.24112970,0.224008423,0.29266542,0.31578947,0.3333333,0.1666667,0.4285714
4,1,3,4,100.61996,-2.395520903,0.22788217,0.493397350,0.16737940,0.26315789,0.3333333,0.1666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,5,104.13405,1.210230525,0.15670738,0.949961205,0.01544487,0.31578947,0.1666667,0.3333333,0.4285714
7,1,6,3,113.64576,-8.031465158,0.33622634,0.943429585,-0.01746381,0.26315789,0.3333333,0.1666667,0.2857143
8,2,1,2,114.96070,30.201924211,0.44823218,0.667831984,0.10864527,0.33333333,0.3333333,0.3333333,0.3333333
9,2,2,5,108.49027,-2.625828192,0.04859062,0.058336892,-0.43016616,0.25000000,0.2857143,0.3333333,0.1428571
10,2,3,2,107.82115,3.649410612,0.63421665,0.289190824,0.25647202,0.26315789,0.3333333,0.1666667,0.2857143
11,2,4,3,97.48834,-3.700316635,0.26393444,0.291736624,0.25516735,0.63157895,0.3333333,0.8333333,0.7142857


In [35]:
table = cbind(lm.list$results, rf.list$results[,-c(1,2)], knn.list$results[,-c(1,2)])
table

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,...,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,2,78.07489,1.4228224,0.6027327,0.0064142416,0.6166746,0.5000000,0.6666667,...,2,137.14728,-14.781768774,0.36015516,0.028612410,-0.50173786,0.26315789,0.1666667,0.3333333,0.2857143
3,1,2,3,80.79492,4.7475213,0.5928941,0.0104225401,0.5870807,0.5000000,0.6666667,...,3,95.09528,2.833708954,0.24112970,0.224008423,0.29266542,0.31578947,0.3333333,0.1666667,0.4285714
4,1,3,1,70.70616,-0.1757042,0.5703419,0.0015566525,0.6892131,0.7222222,0.6666667,...,4,100.61996,-2.395520903,0.22788217,0.493397350,0.16737940,0.26315789,0.3333333,0.1666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,3,93.44264,0.7528499,0.5339227,0.0851030136,0.4170440,0.5000000,0.6666667,...,5,104.13405,1.210230525,0.15670738,0.949961205,0.01544487,0.31578947,0.1666667,0.3333333,0.4285714
7,1,6,3,71.14740,-2.5591976,0.6870041,0.0013743676,0.6947676,0.5000000,0.6666667,...,3,113.64576,-8.031465158,0.33622634,0.943429585,-0.01746381,0.26315789,0.3333333,0.1666667,0.2857143
8,2,1,0,NA,NA,NA,NA,NA,NA,NA,...,2,114.96070,30.201924211,0.44823218,0.667831984,0.10864527,0.33333333,0.3333333,0.3333333,0.3333333
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,...,5,108.49027,-2.625828192,0.04859062,0.058336892,-0.43016616,0.25000000,0.2857143,0.3333333,0.1428571
10,2,3,2,66.70432,-0.4334236,0.6691557,0.0005491549,0.7322905,0.6666667,0.8333333,...,2,107.82115,3.649410612,0.63421665,0.289190824,0.25647202,0.26315789,0.3333333,0.1666667,0.2857143
11,2,4,0,NA,NA,NA,NA,NA,NA,NA,...,3,97.48834,-3.700316635,0.26393444,0.291736624,0.25516735,0.63157895,0.3333333,0.8333333,0.7142857


In [36]:
table.deltas.Lag.3 = table
save(table.deltas.Lag.3, file = 'data/ModelData/models_results_per_cluster_with_delta_lag3_v2.Rdata')

In [37]:
# Clusters that have a linear, a random forest and a knn models
table1 <- table[-which(is.na(table$lm.RMSE)),]
table1

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,...,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,2,78.07489,1.4228224,0.6027327,0.0064142416,0.6166746,0.5000000,0.6666667,...,2,137.14728,-14.781768774,0.3601552,0.02861241,-0.50173786,0.26315789,0.1666667,0.3333333,0.2857143
3,1,2,3,80.79492,4.7475213,0.5928941,0.0104225401,0.5870807,0.5000000,0.6666667,...,3,95.09528,2.833708954,0.2411297,0.22400842,0.29266542,0.31578947,0.3333333,0.1666667,0.4285714
4,1,3,1,70.70616,-0.1757042,0.5703419,0.0015566525,0.6892131,0.7222222,0.6666667,...,4,100.61996,-2.395520903,0.2278822,0.49339735,0.16737940,0.26315789,0.3333333,0.1666667,0.2857143
6,1,5,3,93.44264,0.7528499,0.5339227,0.0851030136,0.4170440,0.5000000,0.6666667,...,5,104.13405,1.210230525,0.1567074,0.94996120,0.01544487,0.31578947,0.1666667,0.3333333,0.4285714
7,1,6,3,71.14740,-2.5591976,0.6870041,0.0013743676,0.6947676,0.5000000,0.6666667,...,3,113.64576,-8.031465158,0.3362263,0.94342958,-0.01746381,0.26315789,0.3333333,0.1666667,0.2857143
10,2,3,2,66.70432,-0.4334236,0.6691557,0.0005491549,0.7322905,0.6666667,0.8333333,...,2,107.82115,3.649410612,0.6342167,0.28919082,0.25647202,0.26315789,0.3333333,0.1666667,0.2857143
12,2,5,1,83.48694,-0.2851609,0.4251210,0.0246503406,0.5269238,0.5000000,0.5000000,...,2,88.99993,-13.954308620,0.2439070,0.06359433,0.43367415,0.42105263,0.6666667,0.1666667,0.4285714
17,4,2,1,101.83925,-1.1833748,0.2370356,0.5718791203,0.1428054,0.2777778,0.6666667,...,2,108.93544,-24.841097403,0.2840779,0.71218176,0.09061509,0.26315789,0.1666667,0.1666667,0.4285714
21,5,1,1,93.11190,-0.3349862,0.2786567,0.1652707713,0.3416376,0.5000000,0.5000000,...,1,138.74172,13.560132173,0.8039915,0.63555778,-0.11624642,0.26315789,0.3333333,0.1666667,0.2857143
22,5,2,2,93.79402,0.1794523,0.3477165,0.1508275560,0.3529395,0.3333333,0.3333333,...,4,87.82113,7.767581174,0.2110068,0.06008094,0.43895592,0.42105263,0.5000000,0.3333333,0.4285714


# Combining all: lags 0, 1, 2 and 3

In [38]:
lm.list <- lm.all(fireSeason_def_2, log1p(masked_ba_series), dates, 
                  cbind(corr.df.delta, corr.df.delta.1[,-c(1,2)], corr.df.delta.2[,-c(1,2)], corr.df.delta.3[,-c(1,2)]), 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 3, useDeltas = T)
lm.list$results

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,lm.acc.t2,lm.acc.t3
2,1,1,7,107.54572,-2.77808843,0.9456270,1.294451e-01,0.37113081,0.6111111,0.6666667,0.6666667,0.5000000
3,1,2,9,82.62976,12.30423354,1.0373864,3.292371e-03,0.65314560,0.5555556,0.5000000,0.5000000,0.6666667
4,1,3,5,85.29747,2.70733451,0.8002434,1.234729e-02,0.57606639,0.5000000,0.5000000,0.3333333,0.6666667
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,7,83.70707,-0.54876414,0.8597187,8.099486e-03,0.60280095,0.4444444,0.6666667,0.3333333,0.3333333
7,1,6,7,60.28585,3.91173249,0.9996329,4.914801e-05,0.80836636,0.6666667,0.8333333,0.5000000,0.6666667
8,2,1,2,83.08598,0.27734702,0.5040408,2.439087e-02,0.54269066,0.5294118,0.5000000,0.4000000,0.6666667
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
10,2,3,8,62.25826,-0.13227957,0.9263980,1.045755e-04,0.78753090,0.5555556,0.6666667,0.3333333,0.6666667
11,2,4,1,91.87663,0.10934779,0.2328123,1.528431e-01,0.35131908,0.4444444,0.6666667,0.3333333,0.3333333


In [39]:
rf.list <- rf.all(fireSeason_def_2, log1p(masked_ba_series), dates, 
                  cbind(corr.df.delta, corr.df.delta.1[,-c(1,2)], corr.df.delta.2[,-c(1,2)], corr.df.delta.3[,-c(1,2)]), 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 3, useDeltas = T)
rf.list$results

,biome,cluster,rf.mtry,rf.Ntree,rf.maxnodes,rf.RMSE,rf.bias,rf.RVar,rf.cor.pvalue,rf.cor,rf.acc,rf.acc.t1,rf.acc.t2,rf.acc.t3
2,1,1,6,175,11,96.36708,-7.40909643,0.12194115,0.38548781,0.2111707729,0.3157895,0.3333333,0.0000000,0.5714286
3,1,2,15,10,13,108.31255,-7.86767162,0.26320976,0.90100486,0.0306089857,0.2105263,0.3333333,0.1666667,0.1428571
4,1,3,1,100,5,106.29293,0.90656760,0.05921384,0.25655010,-0.2738685269,0.3157895,0.3333333,0.5000000,0.1428571
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,7,125,7,100.59320,1.42800296,0.12364724,0.74701962,0.0792679045,0.2631579,0.0000000,0.3333333,0.4285714
7,1,6,15,150,13,121.52695,-0.88215404,0.16525040,0.03569990,-0.4841164428,0.1052632,0.1666667,0.0000000,0.1428571
8,2,1,11,75,5,93.34226,1.07518507,0.16565972,0.22837262,0.2988266414,0.3888889,0.5000000,0.1666667,0.5000000
9,2,2,15,100,11,100.93119,-8.06108276,0.18006342,0.57037586,0.1350060615,0.4500000,0.7142857,0.3333333,0.2857143
10,2,3,15,10,9,97.18499,-8.11787769,0.11954352,0.44252608,0.1873252710,0.3157895,0.1666667,0.3333333,0.4285714
11,2,4,3,50,11,100.00751,-3.54956979,0.07931732,0.85720290,0.0442651927,0.4210526,0.3333333,0.3333333,0.5714286


In [40]:
knn.list <- knn.all(fireSeason_def_2, log1p(masked_ba_series), dates,
                    cbind(corr.df.delta, corr.df.delta.1[,-c(1,2)], corr.df.delta.2[,-c(1,2)], corr.df.delta.3[,-c(1,2)]),
                    list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 3, useDeltas = T)
knn.list$results

,biome,cluster,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,1,160.64356,38.62444828,1.2839244,0.6109866193,-0.124704714,0.2631579,0.0000000,0.3333333,0.4285714
3,1,2,2,89.28628,-5.26571912,0.6897204,0.0248786876,0.512438378,0.4210526,0.6666667,0.1666667,0.4285714
4,1,3,3,106.19568,0.02912691,0.2776248,0.7362239336,0.082766728,0.2631579,0.3333333,0.1666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,4,107.37455,5.73641266,0.1479782,0.7288230420,-0.085174149,0.2105263,0.0000000,0.1666667,0.4285714
7,1,6,4,97.62493,-9.81879391,0.1603421,0.3988653351,0.205415410,0.4736842,0.5000000,0.3333333,0.5714286
8,2,1,1,112.84222,31.46603386,0.7167432,0.2612373850,0.279551088,0.3888889,0.5000000,0.3333333,0.3333333
9,2,2,2,91.98603,0.30904340,0.1789705,0.1415197890,0.340745149,0.6000000,0.8571429,0.5000000,0.4285714
10,2,3,1,98.30708,-7.30437083,0.6936852,0.0831168969,0.407741913,0.4736842,0.3333333,0.1666667,0.8571429
11,2,4,2,94.71441,5.66360946,0.3045276,0.1716646431,0.327081628,0.4210526,0.3333333,0.5000000,0.4285714


In [41]:
table = cbind(lm.list$results, rf.list$results[,-c(1,2)], knn.list$results[,-c(1,2)])
table

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,...,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,7,107.54572,-2.77808843,0.9456270,1.294451e-01,0.37113081,0.6111111,0.6666667,...,1,160.64356,38.62444828,1.2839244,0.6109866193,-0.124704714,0.2631579,0.0000000,0.3333333,0.4285714
3,1,2,9,82.62976,12.30423354,1.0373864,3.292371e-03,0.65314560,0.5555556,0.5000000,...,2,89.28628,-5.26571912,0.6897204,0.0248786876,0.512438378,0.4210526,0.6666667,0.1666667,0.4285714
4,1,3,5,85.29747,2.70733451,0.8002434,1.234729e-02,0.57606639,0.5000000,0.5000000,...,3,106.19568,0.02912691,0.2776248,0.7362239336,0.082766728,0.2631579,0.3333333,0.1666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,7,83.70707,-0.54876414,0.8597187,8.099486e-03,0.60280095,0.4444444,0.6666667,...,4,107.37455,5.73641266,0.1479782,0.7288230420,-0.085174149,0.2105263,0.0000000,0.1666667,0.4285714
7,1,6,7,60.28585,3.91173249,0.9996329,4.914801e-05,0.80836636,0.6666667,0.8333333,...,4,97.62493,-9.81879391,0.1603421,0.3988653351,0.205415410,0.4736842,0.5000000,0.3333333,0.5714286
8,2,1,2,83.08598,0.27734702,0.5040408,2.439087e-02,0.54269066,0.5294118,0.5000000,...,1,112.84222,31.46603386,0.7167432,0.2612373850,0.279551088,0.3888889,0.5000000,0.3333333,0.3333333
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,...,2,91.98603,0.30904340,0.1789705,0.1415197890,0.340745149,0.6000000,0.8571429,0.5000000,0.4285714
10,2,3,8,62.25826,-0.13227957,0.9263980,1.045755e-04,0.78753090,0.5555556,0.6666667,...,1,98.30708,-7.30437083,0.6936852,0.0831168969,0.407741913,0.4736842,0.3333333,0.1666667,0.8571429
11,2,4,1,91.87663,0.10934779,0.2328123,1.528431e-01,0.35131908,0.4444444,0.6666667,...,2,94.71441,5.66360946,0.3045276,0.1716646431,0.327081628,0.4210526,0.3333333,0.5000000,0.4285714


In [42]:
table.deltas.Lag.0123 = table
save(table.deltas.Lag.0123, file = 'data/ModelData/models_results_per_cluster_with_delta_lag0123_v2.Rdata')

# Lag 1, 2, 3

In [43]:
lm.list <- lm.all(fireSeason_def_2, log1p(masked_ba_series), dates, 
                  cbind(corr.df.delta.1, corr.df.delta.2[,-c(1,2)], corr.df.delta.3[,-c(1,2)]), 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 3, useDeltas = T, min.t = 1)
lm.list$results

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,lm.acc.t2,lm.acc.t3
2,1,1,4,78.64337,0.3477003,0.7060449,0.0054973126,0.6255163,0.6666667,0.8333333,0.6666667,0.5000000
3,1,2,7,78.80916,4.6675508,0.9072993,0.0030413958,0.6571934,0.6666667,0.6666667,0.6666667,0.6666667
4,1,3,4,75.07741,-1.1032226,0.7646505,0.0024557025,0.6678315,0.5555556,0.5000000,0.3333333,0.8333333
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,5,75.99792,1.3694093,0.7115569,0.0033571955,0.6521414,0.5000000,0.6666667,0.3333333,0.5000000
7,1,6,6,64.96389,2.7802865,0.9602744,0.0001709848,0.7726204,0.6666667,0.8333333,0.5000000,0.6666667
8,2,1,2,83.08598,0.2773470,0.5040408,0.0243908728,0.5426907,0.5294118,0.5000000,0.4000000,0.6666667
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
10,2,3,6,67.79143,3.0609029,0.8535007,0.0004426202,0.7403204,0.5555556,0.6666667,0.3333333,0.6666667
11,2,4,1,91.87663,0.1093478,0.2328123,0.1528430649,0.3513191,0.4444444,0.6666667,0.3333333,0.3333333


In [44]:
rf.list <- rf.all(fireSeason_def_2, log1p(masked_ba_series), dates, 
                  cbind(corr.df.delta.1, corr.df.delta.2[,-c(1,2)], corr.df.delta.3[,-c(1,2)]), 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 3, useDeltas = T, min.t = 1)
rf.list$results

,biome,cluster,rf.mtry,rf.Ntree,rf.maxnodes,rf.RMSE,rf.bias,rf.RVar,rf.cor.pvalue,rf.cor,rf.acc,rf.acc.t1,rf.acc.t2,rf.acc.t3
2,1,1,5,50,5,78.18139,3.7281584,0.14220567,0.002975784,0.658296646,0.61111111,0.8333333,0.5000000,0.5000000
3,1,2,1,50,9,102.78043,-8.9910783,0.04959079,0.591756396,-0.135550783,0.27777778,0.3333333,0.3333333,0.1666667
4,1,3,10,10,5,100.10478,1.6315536,0.06625681,0.966469533,0.010674227,0.27777778,0.1666667,0.3333333,0.3333333
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,2,75,11,82.89971,-4.3842721,0.13113138,0.015674678,0.559902810,0.50000000,0.5000000,0.3333333,0.6666667
7,1,6,1,100,7,102.53561,-4.7338648,0.03017382,0.353933527,-0.232153486,0.38888889,0.3333333,0.5000000,0.3333333
8,2,1,1,10,9,99.98879,-5.8314810,0.12530561,0.719273377,0.094152123,0.29411765,0.1666667,0.0000000,0.6666667
9,2,2,10,100,7,121.33938,3.0454461,0.12737103,0.007021557,-0.596499277,0.21052632,0.1666667,0.5000000,0.0000000
10,2,3,9,125,5,82.51772,-0.0990742,0.18045260,0.020475917,0.540825572,0.33333333,0.5000000,0.1666667,0.3333333
11,2,4,9,75,7,106.24436,-6.1396716,0.08881418,0.495536922,-0.171768986,0.16666667,0.1666667,0.1666667,0.1666667


In [45]:
knn.list <- knn.all(fireSeason_def_2, log1p(masked_ba_series), dates, 
                  cbind(corr.df.delta.1, corr.df.delta.2[,-c(1,2)], corr.df.delta.3[,-c(1,2)]), 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 3, useDeltas = T, min.t = 1)
knn.list$results

,biome,cluster,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,1,109.57328,2.40073835,1.20327808,0.0786406980,0.425106348,0.4444444,0.8333333,0.1666667,0.3333333
3,1,2,1,85.76602,16.10161119,0.51595074,0.0229951481,0.532197303,0.5555556,0.5000000,0.3333333,0.8333333
4,1,3,5,101.76822,4.06150212,0.05700787,0.7545932130,-0.079251458,0.2777778,0.3333333,0.3333333,0.1666667
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,1,145.48948,1.13809557,1.51179506,0.6636851171,0.110081027,0.3333333,0.5000000,0.3333333,0.1666667
7,1,6,1,133.55689,10.50147684,1.17061097,0.5913743436,0.135689248,0.1666667,0.1666667,0.0000000,0.3333333
8,2,1,2,101.52346,-28.24603579,0.46665489,0.1901376053,0.333985303,0.3529412,0.3333333,0.2000000,0.5000000
9,2,2,1,160.11520,4.66452268,1.03888733,0.1729135417,-0.326183734,0.1052632,0.0000000,0.0000000,0.2857143
10,2,3,5,102.90459,-32.40095729,0.15071932,0.4719263713,0.181146028,0.2222222,0.1666667,0.1666667,0.3333333
11,2,4,1,157.92888,-44.84827287,1.32081094,0.8543481237,-0.046590148,0.2222222,0.0000000,0.1666667,0.5000000


In [46]:
table = cbind(lm.list$results, rf.list$results[,-c(1,2)], knn.list$results[,-c(1,2)])
table

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,...,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,4,78.64337,0.3477003,0.7060449,0.0054973126,0.6255163,0.6666667,0.8333333,...,1,109.57328,2.40073835,1.20327808,0.0786406980,0.425106348,0.4444444,0.8333333,0.1666667,0.3333333
3,1,2,7,78.80916,4.6675508,0.9072993,0.0030413958,0.6571934,0.6666667,0.6666667,...,1,85.76602,16.10161119,0.51595074,0.0229951481,0.532197303,0.5555556,0.5000000,0.3333333,0.8333333
4,1,3,4,75.07741,-1.1032226,0.7646505,0.0024557025,0.6678315,0.5555556,0.5000000,...,5,101.76822,4.06150212,0.05700787,0.7545932130,-0.079251458,0.2777778,0.3333333,0.3333333,0.1666667
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,5,75.99792,1.3694093,0.7115569,0.0033571955,0.6521414,0.5000000,0.6666667,...,1,145.48948,1.13809557,1.51179506,0.6636851171,0.110081027,0.3333333,0.5000000,0.3333333,0.1666667
7,1,6,6,64.96389,2.7802865,0.9602744,0.0001709848,0.7726204,0.6666667,0.8333333,...,1,133.55689,10.50147684,1.17061097,0.5913743436,0.135689248,0.1666667,0.1666667,0.0000000,0.3333333
8,2,1,2,83.08598,0.2773470,0.5040408,0.0243908728,0.5426907,0.5294118,0.5000000,...,2,101.52346,-28.24603579,0.46665489,0.1901376053,0.333985303,0.3529412,0.3333333,0.2000000,0.5000000
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,...,1,160.11520,4.66452268,1.03888733,0.1729135417,-0.326183734,0.1052632,0.0000000,0.0000000,0.2857143
10,2,3,6,67.79143,3.0609029,0.8535007,0.0004426202,0.7403204,0.5555556,0.6666667,...,5,102.90459,-32.40095729,0.15071932,0.4719263713,0.181146028,0.2222222,0.1666667,0.1666667,0.3333333
11,2,4,1,91.87663,0.1093478,0.2328123,0.1528430649,0.3513191,0.4444444,0.6666667,...,1,157.92888,-44.84827287,1.32081094,0.8543481237,-0.046590148,0.2222222,0.0000000,0.1666667,0.5000000


In [47]:
table.deltas.Lag.123 = table
save(table.deltas.Lag.123, file = 'data/ModelData/models_results_per_cluster_with_delta_lag123_v2.Rdata')